In [1]:
from fair_util import BMInterface, BMMetrics
from fair_bm import BMManager
from fair_grid import dummy_model
import pandas as pd
import numpy as np 

C:\Users\enemy\anaconda3\Lib\site-packages\inFairness\utils\ndcg.py:37: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  vect_normalized_discounted_cumulative_gain = vmap(
C:\Users\enemy\anaconda3\Lib\site-packages\inFairness\utils\ndcg.py:48: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  monte_carlo_vect_ndcg = vmap(vect_normalized_disc

In [2]:
wids_train_df = pd.read_csv('./data/wids_pre/wids_train.csv')
wids_val_df = pd.read_csv('./data/wids_pre/wids_val.csv')
wids_test_df = pd.read_csv('./data/wids_pre/wids_test.csv')

In [3]:
# Defining the name of the label column
label_name = 'DiagPeriodL90D'
# Defininf the name of the privileged group
sensitive_attribute = ['patient_age']

bmI = BMInterface(wids_train_df, wids_val_df, wids_test_df, label_name, sensitive_attribute)

In [4]:
x_train, y_train = bmI.get_train_xy() 
x_val , y_val = bmI.get_val_xy()
x_test, y_test = bmI.get_test_xy()

### Model hyperparameter tunning

In [6]:
from catboost import CatBoostClassifier
cat = CatBoostClassifier(eval_metric='Accuracy',
                         depth =  4,
                         learning_rate = 0.01,
                         iterations = 10)

cat.fit(x_train, y_train, eval_set=(x_val, y_val))


0:	learn: 0.8103832	test: 0.8133125	best: 0.8133125 (0)	total: 159ms	remaining: 1.43s
1:	learn: 0.8106061	test: 0.8153926	best: 0.8153926 (1)	total: 169ms	remaining: 677ms
2:	learn: 0.8106061	test: 0.8153926	best: 0.8153926 (1)	total: 180ms	remaining: 420ms
3:	learn: 0.8106061	test: 0.8153926	best: 0.8153926 (1)	total: 189ms	remaining: 284ms
4:	learn: 0.8106061	test: 0.8153926	best: 0.8153926 (1)	total: 199ms	remaining: 199ms
5:	learn: 0.8107175	test: 0.8148726	best: 0.8153926 (1)	total: 211ms	remaining: 141ms
6:	learn: 0.8107175	test: 0.8148726	best: 0.8153926 (1)	total: 223ms	remaining: 95.5ms
7:	learn: 0.8106061	test: 0.8153926	best: 0.8153926 (1)	total: 232ms	remaining: 58ms
8:	learn: 0.8107175	test: 0.8148726	best: 0.8153926 (1)	total: 238ms	remaining: 26.4ms
9:	learn: 0.8106061	test: 0.8153926	best: 0.8153926 (1)	total: 242ms	remaining: 0us

bestTest = 0.8153926157
bestIteration = 1

Shrink model to first 2 iterations.


In [7]:
y_val_pred = cat.predict_proba(x_val)
y_test_pred = cat.predict_proba(x_test)

In [8]:
from sklearn.linear_model import LogisticRegression
import numpy as np

# Training Logistic Regression model
model = LogisticRegression(solver='liblinear', random_state=42)

LR_model = model.fit(x_train, y_train, sample_weight=bmI.get_train_BLD().instance_weights)
y_val_pred = LR_model.predict_proba(x_val)
y_test_pred = LR_model.predict_proba(x_test)

In [9]:
privileged_group = [{'patient_age': 1}]
unprivileged_group = [{'patient_age': 0}]

bmM = BMMetrics(bmI, model.classes_, y_val_pred, y_test_pred, privileged_group, unprivileged_group)

In [10]:
report_dict = bmM.get_report()

print("Balanced Accuracy", report_dict['balanced_acc'])
print("Accuracy", report_dict['acc'])
print("Precision", report_dict['precision'])
print("Recall", report_dict['recall'])
print("F1 Score", report_dict['f1'])
print("MCC", report_dict['mcc'])
print('-----------------------------')    
print("Equal opportunity difference", report_dict['eq_opp_diff'])
print("Average odds difference", report_dict['avg_odd_diff'])
print("Statistical parity difference", report_dict['spd'])
print("Disparate Impact", report_dict['disparate_impact'])
print("Theil index", report_dict['theil_idx'])

Balanced Accuracy 0.7575740824554917
Accuracy 0.8024948024948025
Precision 0.7864619678995115
Recall 0.9383846794338052
F1 Score 0.8557327258921792
MCC 0.2646699883947518
-----------------------------
Equal opportunity difference -0.06223717409587892
Average odds difference -0.06971385825795337
Statistical parity difference -0.13128930817610063
Disparate Impact 0.8499550763701708
Theil index 0.08290776343734235


In [20]:
score_dict = bmM.get_score()
score_dict

0.9374213836477987

In [22]:
bmMR = BMManager(bmI, bmM)

bmMR.pos_ROC()

x_train, y_train = bmI.get_train_xy() 

LR_model = model.fit(x_train, y_train, sample_weight=bmI.get_train_BLD().instance_weights)
y_val_pred = LR_model.predict_proba(x_val)
y_test_pred = LR_model.predict_proba(x_test)

TypeError: BMManager.__init__() missing 1 required positional argument: 'unprivileged_group'

In [ ]:
pred_val_BLD = bmI.get_val_BLD().copy(deepcopy=True)
pred_val_BLD.labels = y_val_pred

pred_BLD = bmI.get_test_BLD().copy(deepcopy=True)
pred_BLD.label = y_test_pred

In [ ]:
bmM.set_new_pred(y_val_pred, y_test_pred)
bmM.get_report()

In [ ]:
score_dict = bmM.get_score()
score_dict

In [ ]:
bmI.restore_BLD()